### 노트북 설명

이 노트북은 사용자 커스텀 전략이 시스템에서 올바르게 작동하는지 검증하기 위한 파일입니다. Google Colab 환경에 맞게 설정되어 있으며, 로컬 환경에서 사용하는 경우 경로 설정에 유의해야 합니다.

**주요 단계:**

1.  **Google Drive 마운트:** Google Drive에 연결하여 필요한 파일에 접근합니다.
2.  **선물 전략 검증:**
    *   전략 파일과 설정 파일을 가져옵니다.(Path 유의)
    *   가져온 전략 및 설정 파일을 사용하여 시스템 데이터 모듈(가짜)을 기반으로 전략을 실행합니다.
    *   전략 실행 결과로 **`"symbol:weights"`** 딕셔너리가 출력되는지 확인합니다.
    *   가중치 절댓값의 합이 1이 넘는지 체크합니다. 1을 초과 할 경우, 전략을 다시 체크하십시오.

**참고:** 사용자 지정 전략을 테스트하려면 해당 전략 파일과 설정 파일의 가져오기 경로를 수정하고 노트북을 실행하십시오.

#### - Original GitHub Link : https://github.com/NeoMatrixAI/nb-runner/tree/main/notebooks

# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


# Verify Futures strategy

In [2]:
# Create Fake module
import sys
import types
from abc import ABC, abstractmethod
from datetime import datetime, timedelta
import requests
import pandas as pd
import numpy as np

# 1. Create a fake module called 'module' and register it with the system.
module_obj = types.ModuleType('module')
sys.modules['module'] = module_obj

# 2. Create a fake empty module called 'module.data_context' and register it with the system.
data_context_module = types.ModuleType('module.data_context')
sys.modules['module.data_context'] = data_context_module

# 3. Define the DataContext class required by the strategy.
class DataContext(ABC):
    @property
    @abstractmethod
    def current_dt(self) -> datetime: pass
    @abstractmethod
    def get_history(self, assets: list, window: int, frequency: str, fields: str or list = 'close') -> pd.DataFrame: pass

# 4. Place the defined DataContext class inside the fake 'module.data_context' module.
data_context_module.DataContext = DataContext

print("✅ Fake 'module.data_context' module and DataContext class created successfully.")

✅ Fake 'module.data_context' module and DataContext class created successfully.


In [ ]:
import sys
import drive.MyDrive.NeoMatrixAI.common as real_common
sys.modules['common'] = real_common

# 1. Import example strategy, config
from drive.MyDrive.NeoMatrixAI.futures.multi_period_momentum.multi_period_momentum import strategy     # Replace with your own strategy file
import drive.MyDrive.NeoMatrixAI.futures.multi_period_momentum.config as config  # Replace with your own config file

# 2. Prepare mock objects and data required for testing.
class MockDataContext(DataContext):
    def __init__(self, fake_data):
        self._fake_data = fake_data

    @property
    def current_dt(self) -> datetime:
        return datetime.now()

    def get_history(self, assets, window, frequency, fields):
        return self._fake_data[fields]

def create_fake_data(assets, num_periods):
    end_time = pd.Timestamp.now(tz='UTC').floor('T')
    dates = pd.date_range(end=end_time, periods=num_periods, freq='1min')

    all_dfs = []
    for i, asset in enumerate(assets):
        trend = 1 + (i - len(assets) / 2) * 0.0001
        base_price = 100 * (1 + i * 0.05)
        prices = np.array([base_price * (trend ** j) for j in range(num_periods)])

        df = pd.DataFrame({
            'datetime': dates,
            'open': prices * (1 + np.random.normal(0, 0.001, num_periods)),
            'high': prices * (1 + np.random.normal(0.002, 0.001, num_periods)),
            'low':  prices * (1 - np.random.normal(0.002, 0.001, num_periods)),
            'close': prices,
            'volume': np.random.uniform(10, 100000, num_periods)
        })
        df['asset'] = asset
        all_dfs.append(df)

    df_concat = pd.concat(all_dfs, ignore_index=True)

    df_final = df_concat.set_index(['asset', 'datetime'])
    df_final = df_final.sort_values(by=['datetime', 'asset'])

    display(df_final)
    return df_final

# 3. Run verify test
test_config = config.strategy_config

fake_data = create_fake_data(
    assets=test_config["assets"], num_periods=1440)
mock_context = MockDataContext(fake_data)

print("\n🚀 Starting strategy function testing...")
results = strategy(context=mock_context, config_dict=test_config)
print("✅ Strategy function execution complete!")

print("\n[Final weighted result]")
results

,,open,high,low,close,volume
asset,datetime,,,,,
ADAUSDT,2025-11-13 04:01:00+00:00,125.072045,125.416932,124.656809,125.000000,70122.276075
BCHUSDT,2025-11-13 04:01:00+00:00,114.880764,115.147879,114.759884,115.000000,84555.202139
BTCUSDT,2025-11-13 04:01:00+00:00,100.027367,100.238039,99.775865,100.000000,37215.001753
ETHUSDT,2025-11-13 04:01:00+00:00,105.133246,105.186862,104.771135,105.000000,74304.244382
LTCUSDT,2025-11-13 04:01:00+00:00,119.956806,120.101870,119.798438,120.000000,51682.274439
...,...,...,...,...,...,...
BCHUSDT,2025-11-14 04:00:00+00:00,115.135444,115.176434,114.676977,115.000000,34352.536495
BTCUSDT,2025-11-14 04:00:00+00:00,64.884187,64.961296,64.795309,64.936211,18968.170165
ETHUSDT,2025-11-14 04:00:00+00:00,78.788625,78.896534,78.563715,78.738447,44752.591844



🚀 Starting strategy function testing...
✅ Strategy function execution complete!

[Final weighted result]


{'ADAUSDT': {'weight': np.float64(0.33410837403540566),
  'presetStopLossPrice': np.float64(158.3474),
  'presetStopSurplusPrice': np.float64(175.0156)},
 'LTCUSDT': {'weight': np.float64(0.16589162596459434),
  'presetStopLossPrice': np.float64(131.6427),
  'presetStopSurplusPrice': np.float64(145.4998)},
 'BTCUSDT': {'weight': np.float64(-0.2488493518707637),
  'presetStopLossPrice': np.float64(68.183),
  'presetStopSurplusPrice': np.float64(61.6894)},
 'ETHUSDT': {'weight': np.float64(-0.16704729724365316),
  'presetStopLossPrice': np.float64(82.6754),
  'presetStopSurplusPrice': np.float64(74.8015)},
 'XRPUSDT': {'weight': np.float64(-0.08410335088558313),
  'presetStopLossPrice': np.float64(100.0193),
  'presetStopSurplusPrice': np.float64(90.4937)}}

In [8]:
abs_sum = sum(abs(info["weight"]) for info in results.values())

if abs_sum > 1:
    print(f"⚠️ Warning: The sum of absolute weights exceeds 1! (Current: {abs_sum:.4f})")
else:
    print(f"✅ OK: Sum of absolute weights = {abs_sum:.4f}")

✅ OK: Sum of absolute weights = 1.0000
